In [1]:
from collections import defaultdict
from pprint import pformat

class SocialProductRecommender:
    def __init__(self, debug=True):
        self.debug = True
        self.iteration = 0

    def log_step(self, step_name, data):
        """Helper to format and print debug information"""
        if self.debug:
            print(f"\n{'='*20} {step_name} {'='*20}")
            print(pformat(data, width=100, indent=2))

    def map_index(self, person, data, current_recommendations):
        """Mapper function that emits both friendship and product data"""
        emitted_data = []
        friendships = data['friends']
        purchases = data['purchases']

        # Emit current direct friendships and purchases
        direct_data = {
            'type': 'direct_data',
            'friends': friendships,
            'purchases': purchases
        }
        emitted_data.append((person, direct_data))

        if self.debug:
            print(f"\nMapper processing {person}:")
            print(f"  Emitting direct data: {direct_data}")

        # Emit friendship and purchase information to friends
        for friend, strength in friendships.items():
            friend_data = {
                'type': 'friend_data',
                'through': person,
                'friendship_strength': strength,
                'original_person': person,
                'purchases': purchases  # Share purchase data with friends
            }
            emitted_data.append((friend, friend_data))

            if self.debug:
                print(f"  Emitting friend data to {friend}: {friend_data}")

        return emitted_data

    def reduce_index(self, person, values):
        """Reducer function that processes friendship and product data"""
        if self.debug:
            print(f"\nReducer processing {person}:")
            print("  Input values:")
            print(pformat(values, indent=4))

        direct_friends = {}
        direct_purchases = {}
        friend_purchases = []

        # First pass: separate direct and friend data
        for value in values:
            if value['type'] == 'direct_data':
                direct_friends = value['friends']
                direct_purchases = value['purchases']
                if self.debug:
                    print(f"  Found direct data:")
                    print(f"    Friends: {direct_friends}")
                    print(f"    Purchases: {direct_purchases}")
            elif value['type'] == 'friend_data':
                friend_purchases.append({
                    'friend': value['original_person'],
                    'strength': value['friendship_strength'],
                    'purchases': value['purchases']
                })
                if self.debug:
                    print(f"  Found friend purchases from {value['original_person']}")

        # Process product recommendations
        product_scores = defaultdict(lambda: {'score': 0, 'recommenders': []})

        for friend_data in friend_purchases:
            friend = friend_data['friend']
            friendship_strength = friend_data['strength']

            for product, rating in friend_data['purchases'].items():
                if product not in direct_purchases:  # Don't recommend products they already have
                    rec = product_scores[product]
                    # Weight the product rating by friendship strength
                    rec['score'] += (rating * friendship_strength) / 100
                    rec['recommenders'].append({
                        'friend': friend,
                        'rating': rating,
                        'friendship_strength': friendship_strength
                    })

        result = {
            'direct_friends': direct_friends,
            'direct_purchases': direct_purchases,
            'product_recommendations': dict(product_scores)
        }

        if self.debug:
            print("\n  Reducer output:")
            print(pformat(result, indent=4))

        return person, result

    def analyze_network(self, social_product_graph):
        """Analyzes the social graph and returns product recommendations"""
        self.log_step("Starting Analysis", social_product_graph)
        recommendations = self.single_mapreduce_pass(social_product_graph)
        return recommendations

    def single_mapreduce_pass(self, social_product_graph, current_recommendations=None):
        """Performs a single MapReduce pass over the social product graph"""
        self.iteration += 1

        if current_recommendations is None:
            current_recommendations = {}

        # MAP phase
        mapped_data = []
        for person, data in social_product_graph.items():
            self.log_step(f"Mapping {person}", data)
            person_mapped_data = self.map_index(person, data, current_recommendations)
            mapped_data.extend(person_mapped_data)

        self.log_step("All Mapped Data", mapped_data)

        # Group data by key for reduce phase
        grouped_data = defaultdict(list)
        for k, v in mapped_data:
            grouped_data[k].append(v)

        self.log_step("Grouped Data", dict(grouped_data))

        # REDUCE phase
        new_recommendations = {}
        for person, values in grouped_data.items():
            self.log_step(f"Reducing {person}", values)
            person, results = self.reduce_index(person, values)
            new_recommendations[person] = results

        self.log_step("Final Recommendations", new_recommendations)

        return new_recommendations



In [2]:
def main():
    # Example social graph with purchase data
    social_product_graph = {
        'Alice': {
            'friends': {'Bob': 90, 'Charlie': 70},
            'purchases': {'Laptop': 95, 'Headphones': 80}
        },
        'Bob': {
            'friends': {'Alice': 90, 'David': 80},
            'purchases': {'Smartphone': 90, 'Headphones': 85, 'Tablet': 75}
        },
        'Charlie': {
            'friends': {'Alice': 70, 'David': 60},
            'purchases': {'Camera': 95, 'Laptop': 70}
        },
        'David': {
            'friends': {'Bob': 80, 'Charlie': 60},
            'purchases': {'Smartphone': 75, 'Camera': 85}
        }
    }

    print("\nAnalyzing social product graph:")
    print(pformat(social_product_graph, indent=2))

    recommender = SocialProductRecommender(debug=True)
    results = recommender.analyze_network(social_product_graph)

    print("\nFinal Product Recommendations Summary:")
    for person, data in sorted(results.items()):
        print(f"\n{person}:")
        print("  Current purchases:")
        for product, rating in sorted(data['direct_purchases'].items()):
            print(f"    - {product} (rated {rating}/100)")

        print("  Recommended products:")
        sorted_recommendations = sorted(
            data['product_recommendations'].items(),
            key=lambda x: x[1]['score'],
            reverse=True
        )
        for product, rec_data in sorted_recommendations:
            print(f"    - {product} (score: {rec_data['score']:.2f})")
            print("      Recommended by:")
            for recommender in rec_data['recommenders']:
                print(f"        * {recommender['friend']} "
                      f"(rated {recommender['rating']}/100, "
                      f"friendship strength: {recommender['friendship_strength']})")

if __name__ == "__main__":
    main()


Analyzing social product graph:
{ 'Alice': { 'friends': {'Bob': 90, 'Charlie': 70},
             'purchases': {'Headphones': 80, 'Laptop': 95}},
  'Bob': { 'friends': {'Alice': 90, 'David': 80},
           'purchases': {'Headphones': 85, 'Smartphone': 90, 'Tablet': 75}},
  'Charlie': { 'friends': {'Alice': 70, 'David': 60},
               'purchases': {'Camera': 95, 'Laptop': 70}},
  'David': { 'friends': {'Bob': 80, 'Charlie': 60},
             'purchases': {'Camera': 85, 'Smartphone': 75}}}

==================== Starting Analysis ====================
{ 'Alice': {'friends': {'Bob': 90, 'Charlie': 70}, 'purchases': {'Headphones': 80, 'Laptop': 95}},
  'Bob': { 'friends': {'Alice': 90, 'David': 80},
           'purchases': {'Headphones': 85, 'Smartphone': 90, 'Tablet': 75}},
  'Charlie': {'friends': {'Alice': 70, 'David': 60}, 'purchases': {'Camera': 95, 'Laptop': 70}},
  'David': {'friends': {'Bob': 80, 'Charlie': 60}, 'purchases': {'Camera': 85, 'Smartphone': 75}}}

=================